# Design of an experiment with TBR using Matched Markets

Please note that this colab is for TBR Geo experiments only.


Using this colab, you can create a geoexperiment design for a client using TBR in combination with Matched Markets. In the following we will use the acronyms TBR for Time Based Regression and MM for Matched Markets. For a general introduction to TBR and MM, please refer to the TBR [paper](https://research.google/pubs/pub45950/), the MM [paper](https://research.google/pubs/pub48983/), and this [introduction](http://www.unofficialgoogledatascience.com/2016/06/estimating-causal-effects-using-geo.html) to geo experiments.

In [ ]:
#@title Load the libraries needed for the design

BAZEL_VERSION = '3.0.0'
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!sudo apt-get install python3-dev python3-setuptools git
!git clone https://github.com/google/matched_markets
!python3 -m pip install ./matched_markets
!pip install colorama
!pip install gspread-dataframe


"""Loading the necessary python modules."""
import altair as alt
import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import re
from scipy import stats

from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

import gspread
import warnings
from colorama import Fore, Style
from gspread_dataframe import set_with_dataframe
from google import auth as google_auth
from google.colab import auth
from google.colab import data_table
from google.colab import widgets
from google.colab import drive
from matched_markets.methodology.common_classes import GeoAssignment
from matched_markets.methodology import geoeligibility
from matched_markets.methodology import tbrmmdata
from matched_markets.methodology import tbrmmdesignparameters
from matched_markets.methodology import tbrmmdiagnostics
from matched_markets.methodology import tbrmatchedmarkets
from matched_markets.methodology import tbrmmdesign
from matched_markets.methodology import utils

warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#@markdown ---
#@markdown ### Enter the trix url for the sheet file containing the Client Sales Data:
#@markdown The spreadsheet should contain the mandatory columns:
#@markdown * date: date in the format YYYY-MM-DD
#@markdown * geo: the number which identifies the geo
#@markdown * response: variable on which you want to measure incrementality
#@markdown (e.g. sales, transactions)
#@markdown * cost: variable used as spend proxy (e.g. ad spend)

#@markdown Other columns can be present in the spreadsheet.

#@markdown Spreadsheet URL containing the geo level response and spend data
client_sales_table = "https://docs.google.com/spreadsheets/d/13AsE5wGq7IKYlwvU_sOkr-XTy02QMcU81O6Kcnhln2g/edit?gid=0#gid=0" #@param {type:"string"}

#@markdown Leave the following field empty if you don't want to add constraint to the geo_eligibility
geo_eligibility_table = "" #@param {type:"string"}
auth.authenticate_user()
creds, _ = google_auth.default()
gc = gspread.authorize(creds)
wks = gc.open_by_url(client_sales_table).sheet1
data = wks.get_all_values()
headers = data.pop(0)
geo_level_time_series = pd.DataFrame(data, columns=headers)

geo_level_time_series["date"] = pd.to_datetime(geo_level_time_series["date"])
for colname in ["response", "geo", "cost"]:
  geo_level_time_series[colname] = pd.to_numeric(geo_level_time_series[colname])

num_geos = geo_level_time_series["geo"].nunique()

if not geo_eligibility_table:
  geo_eligibility = None
else:
  wks = gc.open_by_url(geo_eligibility_table).sheet1
  data = wks.get_all_values()
  headers = data.pop(0)
  geo_eligibility = pd.DataFrame(data, columns=headers)
  for colname in ["geo", "control", "treatment", "exclude"]:
    geo_eligibility[colname] = pd.to_numeric(geo_eligibility[colname])
  # set missing geos in geo_eligibility as eligible for any assignment
  geo_eligibility = utils.default_geo_assignment(geo_level_time_series,
                                                 geo_eligibility)
  geo_eligibility = geoeligibility.GeoEligibility(geo_eligibility)
  geo_eligibility.data.index = pd.to_numeric(geo_eligibility.data.index,
                                             downcast="integer").astype(str)


In [ ]:
def infer_frequency(data: pd.DataFrame, date_index: str,
                    series_index: str) -> str:
    data = data.copy().set_index([series_index, date_index])
    data = data.sort_values(by=[date_index, series_index])
    series_names = data.index.get_level_values(series_index).unique().tolist()
    series_frequencies = []
    for series in series_names:
        observed_times = data.loc[series].index.get_level_values(date_index)
        n_steps = len(observed_times)
        if n_steps > 1:
            # Key fix: Use .days to get integer days instead of dtype conversion
            time_diffs = (observed_times[1:] - observed_times[:-1]).days
            min_frequency = np.min(time_diffs)
            series_frequencies.append(min_frequency)

    if not series_frequencies:
        raise ValueError("At least one series with >1 observation must exist.")

    if not all(freq == series_frequencies[0] for freq in series_frequencies):
        raise ValueError("Inconsistent frequencies across series.")

    frequency_map = {1: 'D', 7: 'W'}
    try:
        return frequency_map[series_frequencies[0]]
    except KeyError:
        raise ValueError(f"Unsupported frequency: {series_frequencies[0]} days.")

In [ ]:
# Copyright 2020 Google LLC.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================
"""TBR Matched Markets preanalysis.
"""
import copy
import itertools
from typing import Generator, List, Set, Text, TypeVar

from matched_markets.methodology import geoeligibility
from matched_markets.methodology import heapdict
from matched_markets.methodology import tbrmmdata
from matched_markets.methodology import tbrmmdesign
from matched_markets.methodology import tbrmmdesignparameters
from matched_markets.methodology import tbrmmdiagnostics
from matched_markets.methodology import tbrmmscore
import numpy as np
import pandas as pd
from scipy import special as scipy_special
# Copyright 2020 Google LLC.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================
"""TBR Matched Markets: Class representing a TBR Design Score.
"""
import collections
from typing import Union
import dataclasses

from matched_markets.methodology import tbrmmdiagnostics

Number = Union[int, float]
TBRMMDiagnostics = tbrmmdiagnostics.TBRMMDiagnostics
Scoring = collections.namedtuple('Scoring', [
    'corr_test', 'aa_test', 'bb_test', 'dw_test', 'corr', 'inv_required_impact'
])


@dataclasses.dataclass
class TBRMMScore:
  """Class representing a TBR design score.

  TBRMMScore defines a score which can be used to sort TBRMMDesign. The scoring
  function is a NamedTuple with properties:

  - corr_test, aa_test, bb_test, dw_test which are True if the corresponding
    diagnostics tests pass. These are prioritize when sorting TBRMMDesign, i.e.
    a design which does not pass one of these tests will be worse than a design
    which pass all of them.

  - corr is the correlation between the current design treatment and
    control group.

  - inv_required_impact is the inverse of the minimum lift which we need to
    generate for a significant result. So, 1/required_impact can be thought of
    as the inverse of the minimum detectable iROAS for a budget of 1$.


  The idea of using the logical result of the tests is due to the fact that it
  could happen that none of the designs searched pass at least 1 of these tests
  (it does not have to be the same tests for all designs). To be constructive,
  we still want to output the best design that can be found, but we will
  highlight/flag the risk of running an experiment which fails one of the tests.
  Knowing which test failed can be useful as well to understand what is "wrong".

  In future, the binary outcome for the tests can be replaced by a p-value or
  similar continuous metrics. For example. the aa test result already provide
  the probability of a false positive result (if the result is positive).

  Attributes:
    diag: a TBRMMDiagnostics object.
  """

  diag: TBRMMDiagnostics  # design diagnostics
  _score = None  # score of the corresponding design

  def __post_init__(self):
    if self.diag.x is None:
      raise ValueError('No Control time series was specified')
    if self.diag.corr is None:
      corr = self.diag.corr
    if self.diag.required_impact is None:
      impact = self.diag.required_impact

  def __lt__(self, other: 'TBRMMScore'):
    return self.score < other.score

  @property
  def score(self):
    if self._score is None:
          """Score of the design."""
      # Handle potential None values in test results
          corr_test = int(self.diag.corr_test) if self.diag.corr_test is not None else 0
          aatest_ok = int(self.diag.aatest.test_ok) if (self.diag.aatest and self.diag.aatest.test_ok is not None) else 0
          bbtest_ok = int(self.diag.bbtest.test_ok) if (self.diag.bbtest and self.diag.bbtest.test_ok is not None) else 0
          dwtest_ok = int(self.diag.dwtest.test_ok) if (self.diag.dwtest and self.diag.dwtest.test_ok is not None) else 0
          corr = round(self.diag.corr, 2) if self.diag.corr is not None else 0.0
          required_impact = self.diag.required_impact if self.diag.required_impact is not None else 1e-6  # Avoid division by zero

          self._score = Scoring(
              corr_test,
              aatest_ok,
              bbtest_ok,
              dwtest_ok,
              corr,
              1 / required_impact
          )
    return self._score

  @score.setter
  def score(self, value: Scoring):
    self._score = value

TBRMMDesignParameters = tbrmmdesignparameters.TBRMMDesignParameters
TBRMMDiagnostics = tbrmmdiagnostics.TBRMMDiagnostics
TBRMMData = tbrmmdata.TBRMMData
TBRMMDesign = tbrmmdesign.TBRMMDesign
TBRMMScore = TBRMMScore
GeoID = Text
GeoIndex = int
DictKey = TypeVar('DictKey', str, int, float)


class TBRMatchedMarkets:
  """TBR Matched Market preanalysis.

  Attributes:
    data: The TBRMMData object.
    parameters: The TBRMMDesignParameters object.
    geo_req_impact: Required minimum incremental impact for each individual geo.
  """

  data: TBRMMData
  parameters: TBRMMDesignParameters
  geo_req_impact: pd.Series

  def __init__(self, data: TBRMMData, parameters: TBRMMDesignParameters):
    """Initialize a TBRMatchedMarkets object.

    Args:
      data: A TBRMMData object.
      parameters: a TBRMMDesignParameters object.
    """

    def estimate_required_impact(y):
      return TBRMMDiagnostics(y,
                              parameters).estimate_required_impact(
                                  parameters.rho_max)
    # Consider only the most recent n_pretest_max time points
    data.df = data.df.iloc[:, -parameters.n_pretest_max:]
    # Calculate the required impact estimates for each geo.
    geo_req_impact = data.df.apply(estimate_required_impact, axis=1)

    self.geo_req_impact = geo_req_impact
    self.data = data
    self.parameters = parameters

  @property
  def geos_over_budget(self) -> Set[GeoID]:
    """Identify geos which do not satisfy the max ad spend budget condition."""
    if self.parameters.budget_range is not None:
      max_impact = self.parameters.budget_range[1] * self.parameters.iroas
      geo_impact = self.geo_req_impact
      geos = set(geo_impact.index[geo_impact > max_impact])
    else:
      geos = set()
    return geos

  @property
  def geos_too_large(self) -> Set[GeoID]:
    """Identify geos which do not satisfy the maximum geo share condition."""
    if self.parameters.treatment_share_range is not None:
      max_trt_share = self.parameters.treatment_share_range[1]
      geo_share = self.data.geo_share
      geos = set(geo_share.index[geo_share > max_trt_share])
    else:
      geos = set()
    return geos

  @property
  def geos_must_include(self) -> Set[GeoID]:
    """Set of geos that must be included in each design."""
    geo_assignments = self.data.geo_eligibility.get_eligible_assignments()
    return geo_assignments.all - geo_assignments.x

  @property
  def geos_within_constraints(self) -> Set[GeoID]:
    """Set of geos that are within the geo-specific constraints.

    Returns:
      Geos that are assignable to control or treatment but not over budget nor
      too large, plus those that must be assigned to the treatment or control
      group (even if over budget or too large). If the maximum number is
      specified, the geos with the highest impact on budget are chosen.
    """
    geos_exceed_size = self.geos_too_large | self.geos_over_budget
    geos = (self.data.assignable - geos_exceed_size) | self.geos_must_include
    n_geos_max = self.parameters.n_geos_max
    if n_geos_max is not None and len(geos) > n_geos_max:
      geos_with_max_impact = list(
          self.geo_req_impact.sort_values(ascending=False).index)
      geos_in_order = list(geo for geo in geos_with_max_impact if geo in geos)
      geos = set(geos_in_order[:n_geos_max])
    return geos

  @property
  def geo_assignments(self) -> geoeligibility.GeoAssignments:
    """Return the possible geo assignments."""

    geos_included = self.geos_within_constraints

    # Order geos in the order of implied budget size ('expensive' first).
    geos_in_order = list(self.geo_req_impact.index)
    geo_index = [geo for geo in geos_in_order if geo in geos_included]

    self.data.geo_index = geo_index
    return self.data.geo_assignments

  def treatment_group_size_range(self) -> range:
    """Range from smallest to largest possible treatment group size."""
    n_treatment_min = max(1, len(self.geo_assignments.t_fixed))
    n_treatment_max = len(self.geo_assignments.t)
    if not self.geo_assignments.cx | self.geo_assignments.c_fixed:
      # No geos left outside the group 't', so reserve at least one geo for the
      # control group.
      n_treatment_max -= 1

    treatment_geos_range = self.parameters.treatment_geos_range
    if treatment_geos_range is None:
      n_geos_from, n_geos_to = (n_treatment_min, n_treatment_max)
    else:
      n_geos_from = max(treatment_geos_range[0], n_treatment_min)
      n_geos_to = min(treatment_geos_range[1], n_treatment_max)

    return range(n_geos_from, n_geos_to + 1)

  def _control_group_size_generator(
      self,
      n_treatment_geos: int) -> Generator[int, None, None]:
    """Acceptable control group sizes, given treatment group size.

    Args:
      n_treatment_geos: Number of treatment geos.

    Yields:
      Control group sizes that agree with the range and ratio constraints.
    """
    n_control_min = max(1, len(self.geo_assignments.c_fixed))
    n_control_max = len(self.geo_assignments.c)

    control_geos_range = self.parameters.control_geos_range
    if control_geos_range is None:
      n_geos_from, n_geos_to = (n_control_min, n_control_max)
    else:
      n_geos_from = max(control_geos_range[0], n_control_min)
      n_geos_to = min(control_geos_range[1], n_control_max)

    if self.parameters.geo_ratio_tolerance is None:
      yield from range(n_geos_from, n_geos_to + 1)
    else:
      geo_tol_max = 1.0 + self.parameters.geo_ratio_tolerance
      geo_tol_min = 1.0 / geo_tol_max
      for n_control_geos in range(n_geos_from, n_geos_to + 1):
        geo_ratio = n_control_geos / n_treatment_geos
        if geo_ratio >= geo_tol_min and geo_ratio <= geo_tol_max:
          yield n_control_geos

  def treatment_group_generator(
      self,
      n: int) -> Generator[Set[GeoIndex], None, None]:
    """Generates all possible treatment groups of given size.

    The indices will generated in the order from smallest to largest, e.g.,
    choosing n=2 geos out of {0, 1, 2, 3} will yield the sequence {0, 1}, {0,
    2}, {0, 3}, {1, 2}, {1, 3}, {2, 3}. The indices refer to geos from largest
    to smallest, i.e., geo 0 is the largest, 4 is the smallest. The fixed geos
    will be added to the set.

    Args:
      n: Size of the treatment group.

    Raises:
      ValueError if n is not positive.

    Yields:
      Sets of geo indices, of length n each. If there are not enough geos
      available, does not yield anything.
    """
    if n <= 0:
      raise ValueError('Treatment group size n must be positive')

    fixed_treatment_geos = self.geo_assignments.t_fixed
    varying_treatment_geos = self.geo_assignments.t - fixed_treatment_geos
    n_remaining = n - len(fixed_treatment_geos)
    if n_remaining == 0 and fixed_treatment_geos:
      yield fixed_treatment_geos  # pytype: disable=bad-return-type
    elif n_remaining > 0:
      it = itertools.combinations(varying_treatment_geos, n_remaining)
      for treatment_geos_combination in it:
        treatment_group = fixed_treatment_geos | set(treatment_geos_combination)
        yield treatment_group  # pytype: disable=bad-return-type

  def control_group_generator(
      self,
      treatment_group: Set[GeoIndex]) -> Generator[Set[GeoIndex], None, None]:
    """Iterates over control geo combinations, given a treatment group.

    Args:
      treatment_group: Set of treatment geos. The sequence of control groups is
      constructed from the remaining geos.

    Yields:
      Sets of geo indices representing control groups. If there are not enough
      geos available, does not yield anything.
    """
    if not treatment_group:
      raise ValueError('Treatment group must not be empty')

    # The treatment group must be a subset of the available treatment geos.
    invalid_geo_indices = treatment_group - self.geo_assignments.t
    if invalid_geo_indices:
      geos = [str(geo_index) for geo_index in sorted(invalid_geo_indices)]
      raise ValueError(
          'Invalid treatment geo indices: ' + ', '.join(geos))

    # The fixed control geos are those that belong to either 'c_fixed' or 'ct'.
    # The geos in the group 'ct' must be assigned to control or treatment, but
    # not excluded.
    ct_geos_not_in_treatment = self.geo_assignments.ct - treatment_group
    fixed_control_geos = self.geo_assignments.c_fixed | ct_geos_not_in_treatment
    possible_control_geos = self.geo_assignments.c - treatment_group

    # The 'varying control geos' can be in the groups 'cx' or 'ctx' only.
    varying_control_geos = possible_control_geos - fixed_control_geos

    n_treatment_geos = len(treatment_group)

    for n_control_geos in self._control_group_size_generator(n_treatment_geos):
      n_remaining = n_control_geos - len(fixed_control_geos)
      if n_remaining == 0 and fixed_control_geos:
        yield fixed_control_geos  # pytype: disable=bad-return-type
      elif n_remaining > 0:
        # If n_remaining > len(varying_control_geos), the generator will not
        # return anything.
        it = itertools.combinations(varying_control_geos, n_remaining)
        for control_geos in it:
          control_group = fixed_control_geos | set(control_geos)
          yield control_group  # pytype: disable=bad-return-type

  def count_max_designs(self) -> int:
    """Count (fast) how many designs there are at most.

    Only the group sizes and their ratio is used as a constraint.

    Returns:
      Maximum number of designs under the constraint of the geo eligibility
      matrix, and the geo group sizes and allowed ratios.
    """
    n_t_fixed = len(self.geo_assignments.t_fixed)
    n_c_fixed = len(self.geo_assignments.c_fixed)
    n_cx = len(self.geo_assignments.cx)
    n_tx = len(self.geo_assignments.tx)
    n_ct = len(self.geo_assignments.ct)
    n_ctx = len(self.geo_assignments.ctx)
    trt_sizes = set(self.treatment_group_size_range())
    # Pre-compute the control sizes to avoid repetition within the loop.
    control_group_sizes = {}
    for n_trt in trt_sizes:
      group_sizes = set(self._control_group_size_generator(n_trt))
      control_group_sizes[n_trt] = group_sizes
    n_designs = 0
    # Split the space into the subspaces cx, tx, ct, ctx.
    comb = scipy_special.comb
    for i_ct in range(1 + n_ct):
      n1 = comb(n_ct, i_ct, exact=True)
      for i_tx in range(1 + n_tx):
        n2 = comb(n_tx, i_tx, exact=True)
        for i_ctx in range(1 + n_ctx):
          n_trt = n_t_fixed + i_tx + i_ctx + i_ct
          if n_trt in trt_sizes:
            ctl_sizes = control_group_sizes[n_trt]
            n3 = comb(n_ctx, i_ctx, exact=True)
            for i_cx in range(1 + n_cx):
              n4 = comb(n_cx, i_cx, exact=True)
              for i_cctx in range(1 + n_ctx - i_ctx):
                n_ctl = n_c_fixed + i_cx + i_cctx + (n_ct - i_ct)
                if n_ctl in ctl_sizes:
                  n5 = comb(n_ctx - i_ctx, i_cctx, exact=True)
                  n_designs += n1 * n2 * n3 * n4 * n5
    return n_designs

  def exhaustive_search(self) -> List[TBRMMDesign]:
    """Search the design space for acceptable designs, within the constraints.

    Returns:
      the set of feasible designs found given the design parameters,
        with their corresponding treatment/control groups and score.
    """
    treatment_share_range = self.parameters.treatment_share_range
    budget_range = self.parameters.budget_range

    # Do not store patterns when we have the last treatment pattern size.
    skip_this_trt_group_size = list(self.treatment_group_size_range()).pop()
    skip_treatment_geo_patterns = []

    results = heapdict.HeapDict(size=self.parameters.n_designs)

    def skip_if_subset(geos: Set[GeoIndex]) -> bool:
      """Check if one of the stored geo patterns is a subset of the geos.

      Args:
        geos: Set of geo indices.

      Returns:
        bool: True if one of the stored groups is a subset of the geos.
      """
      for p in skip_treatment_geo_patterns:
        if set(p).issubset(geos):
          return True
      return False

    volume_tol = self.parameters.volume_ratio_tolerance
    if volume_tol is not None:
      tol_min = 1.0 / (1.0 + volume_tol)
      tol_max = 1.0 + volume_tol

    treatment_group_sizes = self.treatment_group_size_range()
    for treatment_group_size in treatment_group_sizes:

      # Treatment groups are saved for the purpose of the inclusion check.
      save_treatment_groups = (treatment_group_size != skip_this_trt_group_size)

      treatment_groups = self.treatment_group_generator(treatment_group_size)
      for treatment_group in treatment_groups:
        treatment_share = self.data.aggregate_geo_share(treatment_group)
        if treatment_share_range is not None:
          # Skip this treatment group if the group implies too low or high share
          # of response volume.
          if (treatment_share > treatment_share_range[1] or
              treatment_share < treatment_share_range[0]):
            continue
        elif skip_if_subset(treatment_group):
          # If the group is a superset of a group that we already know has too
          # high a share or budget, then skip this group too.
          continue
        y = self.data.aggregate_time_series(treatment_group)
        diag = TBRMMDiagnostics(y, self.parameters)
        req_impact = diag.estimate_required_impact(self.parameters.rho_max)
        req_budget = req_impact / self.parameters.iroas
        if budget_range is not None:
          # If the budget is too high, skip this treatment group.
          if req_budget > budget_range[1]:
            if save_treatment_groups:
              # We skip all treatment groups that are a superset of a treatment
              # group that has too high an estimated budget.
              skip_treatment_geo_patterns.append(treatment_group)
              continue
            # If the budget is too low, skip this treatment group.
          elif req_budget < budget_range[0]:
            continue
        control_groups = self.control_group_generator(treatment_group)
        for control_group in control_groups:
          if volume_tol is not None:
            control_share = self.data.aggregate_geo_share(control_group)
            xy_share = control_share / treatment_share
            if xy_share > tol_max or xy_share < tol_min:
              continue
          diag.x = self.data.aggregate_time_series(control_group)
          corr = diag.corr  # pylint: disable=unused-variable
          req_impact = diag.required_impact
          req_budget = req_impact / self.parameters.iroas
          if (budget_range is not None and (self._constraint_not_satisfied(
              req_budget, budget_range[0], budget_range[1]))):
            continue

          # deepcopy is needed otherwise diag.corr gets overwritten, and so
          # it will not be equal to diag.score.score.corr for some reason
          design_score = TBRMMScore(copy.deepcopy(diag))
          score = design_score.score
          if budget_range is not None:
            # If the budget was specified then we use the inverse of the
            # minimum detectable iROAS for the max. budget as the last value
            # in the scoring, instead of using the same for a budget of 1$
            iroas = req_impact / budget_range[1]
            design_score.score = score._replace(inv_required_impact=1 / iroas)

          # deepcopy is needed otherwise diag.corr gets overwritten, and so
          # it will not be equal to diag.score.score.corr for some reason
          design = TBRMMDesign(
              design_score, treatment_group, control_group,
              copy.deepcopy(diag))
          results.push(0, design)

    self._search_results = results
    return self.search_results()

  def search_results(self):
    """Outputs the results of the exhaustive search in a friendly format.

    Returns:
      results: the set of feasible designs found given the design parameters,
        with their corresponding treatment/control groups and score.

    """
    result = self._search_results.get_result()
    output_result = []
    if result:
      design = result[0]
      # map from geo indices to geo IDs.
      for d in design:
        treatment_geos = {self.data.geo_index[x] for x in d.treatment_geos}
        control_geos = {self.data.geo_index[x] for x in d.control_geos}
        d.treatment_geos = treatment_geos
        d.control_geos = control_geos
        output_result.append(d)

    return output_result

  @staticmethod
  def _constraint_not_satisfied(parameter_value: float,
                                constraint_lower: float,
                                constraint_upper: float) -> bool:
    """Checks if the parameter value is in the interval [constraint_lower, constraint_upper]."""
    return (parameter_value < constraint_lower) | (
        parameter_value > constraint_upper)

  def design_within_constraints(self, treatment_geos: Set[GeoIndex],
                                control_geos: Set[GeoIndex]):
    """Checks if a set of control/treatment geos passes all constraints.

    Given a set of control and treatment geos we verify that some of their
    metrics are within the bounds specified in TBRMMDesignParameters.

    Args:
      treatment_geos: Set of geo indices referring to the geos in treatment.
      control_geos: Set of geo indices referring to the geos in control.

    Returns:
      False if any specified constraint is not satisfied.
    """
    if self.parameters.volume_ratio_tolerance is not None:
      volume_ratio = (
          self.data.aggregate_geo_share(control_geos)/
          self.data.aggregate_geo_share(treatment_geos))
      if self._constraint_not_satisfied(
          volume_ratio, 1 / (1 + self.parameters.volume_ratio_tolerance),
          1 + self.parameters.volume_ratio_tolerance):
        return False

    if self.parameters.geo_ratio_tolerance is not None:
      geo_ratio = len(control_geos) / len(treatment_geos)
      if self._constraint_not_satisfied(
          geo_ratio, 1 / (1 + self.parameters.geo_ratio_tolerance),
          1 + self.parameters.geo_ratio_tolerance):
        return False

    if self.parameters.treatment_share_range is not None:
      treatment_response_share = self.data.aggregate_geo_share(
          treatment_geos) / self.data.aggregate_geo_share(
              self.geo_assignments.all)
      if self._constraint_not_satisfied(
          treatment_response_share, self.parameters.treatment_share_range[0],
          self.parameters.treatment_share_range[1]):
        return False

    if self.parameters.treatment_geos_range is not None:
      num_treatment_geos = len(treatment_geos)
      if self._constraint_not_satisfied(
          num_treatment_geos, self.parameters.treatment_geos_range[0],
          self.parameters.treatment_geos_range[1]):
        return False

    if self.parameters.control_geos_range is not None:
      num_control_geos = len(control_geos)
      if self._constraint_not_satisfied(num_control_geos,
                                        self.parameters.control_geos_range[0],
                                        self.parameters.control_geos_range[1]):
        return False

    return True

  def greedy_search(self):
    """Searches the Matched Markets for a TBR experiment.

    Uses a greedy hill climbing algorithm to provide recommended 'matched
    markets' experimental group assignments that appear to lead to valid and
    effective TBR models relative to the pretest period.  This is accomplished
    by using a greedy hill climbing alogirhtm that alternates between two
    routines:
    1) Looks for the best set of control geos given the current
       set of treatment geos.
    2) Adds one new geo to the set of treatment geos given
       the current control group.

    See Au (2018) for more details.

    Returns:
      the set of feasible designs found given the design parameters,
        with their corresponding treatment/control groups and score.
    """
    budget_range = self.parameters.budget_range
    results = heapdict.HeapDict(size=self.parameters.n_designs)

    if self.parameters.treatment_geos_range is None:
      n_treatment = len(self.geo_assignments.t)
      max_treatment_size = n_treatment
      n_remaining = len(self.geo_assignments.all) - n_treatment
      if n_remaining == 0:
        max_treatment_size = n_treatment - 1
      self.parameters.treatment_geos_range = (1, max_treatment_size)
    else:
      max_treatment_size = self.parameters.treatment_geos_range[1]

    if self.parameters.control_geos_range is None:
      n_control = len(self.geo_assignments.c)
      max_control_size = n_control
      n_remaining = len(self.geo_assignments.all) - n_control
      if n_remaining == 0:
        max_control_size = n_control - 1
      self.parameters.control_geos_range = (1, max_control_size)

    kappa_0 = len(self.geo_assignments.t_fixed)
    group_star_trt = {kappa_0: self.geo_assignments.t_fixed}
    tmp_diag = TBRMMDiagnostics(np.random.normal(range(100)), self.parameters)
    tmp_diag.x = list(range(len(tmp_diag.y)))
    tmp_score = TBRMMScore(tmp_diag)
    tmp_score.score = tmp_score.score._replace(
        corr_test=0,
        aa_test=0,
        bb_test=0,
        dw_test=0,
        corr=0,
        inv_required_impact=0)
    score_star = {kappa_0: tmp_score}
    group_ctl = self.geo_assignments.c
    if kappa_0 == 0:
      group_star_ctl = {kappa_0: group_ctl}
      needs_matching = False
    else:
      group_star_ctl = {}
      needs_matching = True

    k = kappa_0
    while (k < max_treatment_size) | (needs_matching):
      # Find the best control group given the current treatment group
      if needs_matching:
        r_control = self.geo_assignments.c - (group_ctl | group_star_trt[k])
        r_unassigned = (group_ctl & self.geo_assignments.x) - group_star_trt[k]

        reassignable_geos = r_control | r_unassigned
        treatment_time_series = self.data.aggregate_time_series(
            group_star_trt[k])
        current_design = TBRMMDiagnostics(treatment_time_series,
                                          self.parameters)
        current_design.x = self.data.aggregate_time_series(group_ctl)
        current_score = TBRMMScore(current_design)

        group_ctl_tmp = group_ctl
        for geo in reassignable_geos:
          neighboring_control_group = group_ctl.symmetric_difference([geo])
          # we skip checking constraints for designs with less than the minimum
          # number of treatment geos, or above the maximum number of control
          # geos. Otherwise, we will never be able to augment the size of
          # treatment (to reach a size which would pass the checks) or decrease
          # the size of control
          if (k >= self.parameters.treatment_geos_range[0]) and (
              len(neighboring_control_group) <=
              self.parameters.control_geos_range[1]):
            if (not neighboring_control_group) or (
                not self.design_within_constraints(group_star_trt[k],
                                                   neighboring_control_group)):  # pytype: disable=wrong-arg-types
              continue

          neighbor_design = tbrmmdiagnostics.TBRMMDiagnostics(
              treatment_time_series, self.parameters)
          neighbor_design.x = self.data.aggregate_time_series(
              neighboring_control_group)
          req_impact = neighbor_design.required_impact
          req_budget = req_impact / self.parameters.iroas
          if (budget_range is not None) and (self._constraint_not_satisfied(
              req_budget, budget_range[0], budget_range[1])):
            continue

          score = TBRMMScore(neighbor_design)
          if score > current_score:
            group_ctl_tmp = neighboring_control_group
            current_score = score

        if current_score > TBRMMScore(current_design):
          group_ctl = group_ctl_tmp
        else:
          group_star_ctl[k] = group_ctl_tmp
          score_star[k] = current_score
          needs_matching = False
      # add one geo to treatment given the current control group
      elif k < max_treatment_size:
        r_treatment = self.geo_assignments.t - group_star_trt[k]

        current_score = copy.deepcopy(tmp_score)
        group_trt = group_star_trt[k]
        for geo in r_treatment:
          augmented_treatment_group = group_star_trt[k].union([geo])
          updated_control_group = group_star_ctl[k] - set([geo])
          # see comment on lines 566-567 for the same if statement
          if (k >= self.parameters.treatment_geos_range[0]) and (
              len(updated_control_group) <=
              self.parameters.control_geos_range[1]):
            if (not updated_control_group) or (
                not self.design_within_constraints(augmented_treatment_group,
                                                   updated_control_group)):
              continue
          treatment_time_series = self.data.aggregate_time_series(
              augmented_treatment_group)
          neighbor_design = TBRMMDiagnostics(
              treatment_time_series, self.parameters)
          neighbor_design.x = self.data.aggregate_time_series(
              updated_control_group)
          req_impact = neighbor_design.required_impact
          req_budget = req_impact / self.parameters.iroas
          if (budget_range is not None) and (self._constraint_not_satisfied(
              req_budget, budget_range[0], budget_range[1])):
            continue
          score = TBRMMScore(neighbor_design)
          if score > current_score:
            group_ctl = updated_control_group
            group_trt = augmented_treatment_group
            current_score = score

        group_star_trt[k+1] = group_trt
        k = k + 1
        needs_matching = True

    # if some geos are fixed to treatment, we did not check that the design
    # with treatment group = {all geos fixed in treatment} and control group =
    # {all geos that can be assigned to control} pass the diagnostic tests
    if kappa_0 > 0:
      diagnostic = TBRMMDiagnostics(
          self.data.aggregate_time_series(group_star_trt[kappa_0]),
          self.parameters)
      diagnostic.x = self.data.aggregate_time_series(group_star_ctl[kappa_0])
      req_impact = diagnostic.required_impact
      req_budget = req_impact / self.parameters.iroas
      if (not group_star_ctl[kappa_0]) or (not self.design_within_constraints(
          group_star_trt[kappa_0], group_star_ctl[kappa_0])):
        if (budget_range is not None) and (self._constraint_not_satisfied(
            req_budget, budget_range[0], budget_range[1])):
          group_star_trt.pop(kappa_0, None)
          group_star_ctl.pop(kappa_0, None)
          score_star.pop(kappa_0, None)

    group_star_trt.pop(0, None)
    group_star_ctl.pop(0, None)
    score_star.pop(0, None)
    for k in group_star_trt:
      if self.design_within_constraints(group_star_trt[k], group_star_ctl[k]):
        design_diag = TBRMMDiagnostics(
            self.data.aggregate_time_series(group_star_trt[k]), self.parameters)
        design_diag.x = self.data.aggregate_time_series(group_star_ctl[k])
        design_score = TBRMMScore(design_diag)
        design = TBRMMDesign(
            design_score, group_star_trt[k], group_star_ctl[k],
            copy.deepcopy(design_diag))
        results.push(0, design)

    self._search_results = results
    return self.search_results()


In [ ]:
#@title Select the parameters for the design of the experiment

## The minimum detectable iROAS is defined as the value of the true iROAS such
## that, given a confidence_level (input) % confidence level for a one-sided
## test, gives a power_level (input) % power if the true iROAS is equal to the
## minimum detectable iROAS.
minimum_detectable_iROAS =  3#@param{type: "number"}
#@markdown Use an average order value of 1 if the design is based on
#@markdown sales/revenue or an actual average order value (e.g. $80) for a
#@markdown design based on transactions/footfall/contracts.
average_order_value =  1#@param{type: "number"}

confidence_level = 0.90 #@param {type:"number"}
power_level = 0.80 #@param {type:"number"}
experiment_duration_in_weeks = 4 #@param {type:"integer"}

#@markdown List the maximum budget for the experiment e.g. 300000
experiment_budget =  300000#@param{type: "number"}
#@markdown List any alternative budget which you would like to test separated
#@markdown by a comma, e.g. 125000, 150000
alternative_budget = "" #@param{type: "string"}
additional_budget = [float(re.sub(r"\W+", "", x)) for x in
                     alternative_budget.split(',') if alternative_budget != ""]

#@markdown List the days and time periods that you want to exclude separated by
#@markdown a comma e.g. 2019/10/10, 2010/10/11, 2018/10/20-2018/11/20.
#@markdown The format for time periods is "YYYY/MM/DD - YYYY/MM/DD",
#@markdown where the two dates specify the start and end date for the period.
#@markdown The format for day is "YYYY/MM/DD". Leave empty to
#@markdown use all days/weeks.
day_week_exclude = "" #@param {type: "string"}
day_week_exclude = [] if day_week_exclude == "" else [
    re.sub(r"\s+", "", x) for x in day_week_exclude.split(",")
]
## Find all the days we should exclude from the analysis from the input
periods_to_exclude = utils.find_days_to_exclude(day_week_exclude)
days_exclude = utils.expand_time_windows(periods_to_exclude)

## Additional constraints which will be flagged in red if not met in
## the design

# upper bound on the minimal detectable relative lift
minimum_detectable_lift_in_response_metric = 0.1 * 100
# lower bound on the baseline revenue covered by the treatment group
minimum_revenue_covered_by_treatment = 0.05 * 100



frequency = infer_frequency(geo_level_time_series, 'date', 'geo')
if frequency == "D":
  n_test = experiment_duration_in_weeks * 7
  ## Use the most recent ~6 months
  n_pretest = 180
elif frequency == "W":
  n_test = experiment_duration_in_weeks
  n_pretest = 26

## Other constraints/parameters which are hidden to the user

## Ratio of avg. control group response / avg. treatment group response must be
## between 1/(1+volume_ratio_tolerance) and 1+volume_ratio_tolerance
volume_ratio_tolerance = np.inf
## Ratio of number of control geos / number treatment geos must be
## between 1/(1+geo_ratio_tolerance) and 1+geo_ratio_tolerance
geo_ratio_tolerance = np.inf
## Constrain on the % of the treatment group response with respect to the
## overall response
treatment_share_range = (0.0001, 0.9999)
## Minimum and maximum number of geos to include in the treatment group
treatment_geos_range = (1, num_geos - 1)
## Minimum and maximum number of geos to include in the control group
control_geos_range = (1, num_geos - 1)
## Maximum number of geos to include in the search
n_geos_max = num_geos
## Maximum number of pretest timepoints to include in the time series for the
## purpose of estimating minimum detectable response
n_pretest_max = n_pretest
## Number of design to store during the exhaustive search
n_designs = 3
## Maximum assumed treatment-control correlation to use for estimating the MDR
rho_max = 0.995
## Minimum acceptable Pearson correlation between the treatment and control
## time series.
min_corr = 0.8
## Inverse quantile of the f distribution parameter 'phi' used in the TBR
## preanalysis formula.
flevel = 0.9

budget_range = (0.1, experiment_budget)
min_volume_ratio = 1/(1 + volume_ratio_tolerance)
max_volume_ratio = 1 + volume_ratio_tolerance

tbr_parameters = tbrmmdesignparameters.TBRMMDesignParameters(
    n_test=n_test,
    iroas=minimum_detectable_iROAS,
    volume_ratio_tolerance=volume_ratio_tolerance,
    geo_ratio_tolerance=geo_ratio_tolerance,
    treatment_share_range=treatment_share_range,
    budget_range=budget_range,
    treatment_geos_range=treatment_geos_range,
    control_geos_range=control_geos_range,
    n_geos_max=n_geos_max,
    n_pretest_max=n_pretest_max,
    n_designs=n_designs,
    sig_level=confidence_level,
    power_level=power_level,
    min_corr=min_corr,
    rho_max=rho_max,
    flevel=flevel)

# remove dates that the user wants to exclude
data_for_design = geo_level_time_series[~geo_level_time_series['date']
                                        .isin(days_exclude)].copy()
tbrclass = tbrmmdata.TBRMMData(df=data_for_design,
                               response_column='response',
                               geo_eligibility=geo_eligibility)

MMclass = TBRMatchedMarkets(data=tbrclass,
                                              parameters=tbr_parameters)

In [ ]:
#@title Summary of the possible designs

max_feasible_number_of_designs = 5 * 10 ** 6

if MMclass.count_max_designs() < max_feasible_number_of_designs:
  matched_designs = MMclass.exhaustive_search()
else:
  matched_designs = MMclass.greedy_search()

if len(matched_designs) == 0:
  raise ValueError(f'{Fore.RED}It wasn\'t possible to find a design within ' +
                   f'the constraint in input or because all the designs, ' +
                   f'did not pass one among the AA test, structural break ' +
                   f'test, or minimum correlation of 0.8\n{Style.RESET_ALL}')

matched_designs.sort(reverse=True)
chosen_design = matched_designs[0]

if not chosen_design.score.score.aa_test:
  print(f'{Fore.RED}WARNING: the design does not pass the A/A test. ' +
        f'We do not recommend to use the proposed design.\n{Style.RESET_ALL}')

if not chosen_design.score.score.bb_test:
  print(f'{Fore.RED}WARNING: the design does not pass the Brownian Bridge ' +
        f'test. The relationship between treatment/control is ' +
        f'not stable over time. We do not recommend to use ' +
        f'the proposed design.\n{Style.RESET_ALL}')

if not chosen_design.score.score.dw_test:
  print(f'{Fore.RED}WARNING: the design does not pass the Durbin-Watson ' +
        f'test. The residuals are autocorrelated. We do not recommend to use ' +
        f'the proposed design.\n{Style.RESET_ALL}')

minimum_iroas_aov = minimum_detectable_iROAS / average_order_value
minimum_detectable_impact = chosen_design.diag.estimate_required_impact(
    chosen_design.diag.corr)

optimal_budget = minimum_detectable_impact / minimum_iroas_aov
lower_budget = optimal_budget *  0.8
upper_budget = optimal_budget * 1.2
list_of_budgets = [lower_budget, optimal_budget, upper_budget
                  ] + additional_budget

first_day = geo_level_time_series["date"].max() - pd.Timedelta(
    str(experiment_duration_in_weeks) + "W")
most_recent_geo_level_time_series = geo_level_time_series[
    geo_level_time_series['date'] > first_day]

total_response = most_recent_geo_level_time_series["response"].sum()
total_spend = most_recent_geo_level_time_series["cost"].sum()
chosen_design.treatment_geos = {int(x) for x in chosen_design.treatment_geos}
chosen_design.control_geos = {int(x) for x in chosen_design.control_geos}
designs = []
for budget in list_of_budgets:
  baseline = most_recent_geo_level_time_series.loc[
      most_recent_geo_level_time_series["geo"].isin(chosen_design.treatment_geos
                                                   ), "response"].sum()
  cost_in_experiment = most_recent_geo_level_time_series.loc[
      most_recent_geo_level_time_series["geo"].isin(chosen_design.treatment_geos
                                                   ), "cost"].sum()
  min_detectable_iroas = (
      average_order_value * minimum_detectable_impact / budget)
  min_detectable_lift = (minimum_detectable_impact * 100 / baseline)
  num_treatment_geos = len(chosen_design.treatment_geos)
  num_control_geos = len(chosen_design.control_geos)
  num_removed_geos = num_geos - num_treatment_geos - num_control_geos
  treat_control_removed = (f'{num_treatment_geos}  /  {num_control_geos}  / ' +
                           f'{num_removed_geos}')
  revenue_covered = 100 * baseline / total_response
  proportion_cost_in_experiment = cost_in_experiment / total_spend
  national_budget = utils.human_readable_number(
      budget / proportion_cost_in_experiment)
  designs.append({
      "Budget": utils.human_readable_number(budget),
      "Minimum detectable iROAS": f'{min_detectable_iroas:.3}',
      "Minimum detectable lift in response": f'{min_detectable_lift:.2f} %',
      "Treatment/control/excluded geos": treat_control_removed,
      "Revenue covered by treatment group": f'{revenue_covered:.2f} %',
      "Cost/baseline response": f'{(budget / baseline * 100):.2f} %',
      "Cost if test budget is scaled nationally": national_budget
  })


## define function to colorcode rows and cells of the output table
def is_optimal_design(row):
    """Color a row in:
    - orange if the minimum detectable iROAS of the corresponding design
      is larger than the target
    - green if its equal to the target
    - beige if it's lower than the target
    """
    if float(row["Minimum detectable iROAS"]) == minimum_detectable_iROAS:
          return pd.Series('background-color: lightgreen', row.index)
    elif float(row["Minimum detectable iROAS"]) > minimum_detectable_iROAS:
          return pd.Series('background-color: orange', row.index)
    else:
          return pd.Series('background-color: beige', row.index)

def flag_warning_lift(val, value):
    """
    Color a cell in red if its value is larger than the value
    in input
    """
    color = 'red' if float(val.strip(' %')) > value else 'black'
    return 'color: %s' % color

def flag_warning_revenue(val, value):
    """
    Color a cell in red if its value is smaller than the value
    in input
    """
    color = 'red' if float(val.strip(' %')) < value else 'black'
    return 'color: %s' % color


## convert the table to a pd.DataFrame and select a subset of columns
designs = pd.DataFrame(designs)
designs.index.rename("Design", inplace=True)
designs = designs[["Budget", "Minimum detectable iROAS",
                   "Minimum detectable lift in response",
                   "Treatment/control/excluded geos",
                   "Revenue covered by treatment group",
                   "Cost/baseline response",
                   "Cost if test budget is scaled nationally"]]

## apply colorcoding to rows and cells of the table
designs_table = designs.style.applymap(
    flag_warning_lift,
    value=minimum_detectable_lift_in_response_metric,
    subset=["Minimum detectable lift in response"]).applymap(
        flag_warning_revenue,
        value=minimum_revenue_covered_by_treatment,
        subset=["Revenue covered by treatment group"]).apply(
            is_optimal_design, axis=1)

designs_table


In [ ]:
#@title Select the design to be used in the experiment
#@markdown Select the design using the number as displayed in the table in
#@markdown the cell called "Summary of the possible designs".

selected_design =   1#@param {type:"integer"}

if selected_design not in designs.index:
  raise ValueError(f'the selected design must be one of {designs.index.to_list()}, got {selected_design}')

selected_design = int(selected_design)
final_design = designs[designs.index == selected_design]
selected_budget = final_design["Budget"].values[0]

# these are numerical identifier used to denote the two groups
group_treatment = GeoAssignment.TREATMENT
group_control = GeoAssignment.CONTROL
group_excluded = GeoAssignment.EXCLUDED

data_for_design.loc["assignment"] = group_excluded
data_for_design.loc[data_for_design["geo"].isin(
    chosen_design.control_geos), "assignment"] = group_control
data_for_design.loc[data_for_design["geo"].isin(
    chosen_design.treatment_geos), "assignment"] = group_treatment

# set basic plot parameters
chart_width = 600
chart_height = 300
axis_title_font_size = 16
axis_label_font_size = 14
title_font_size = 19

plot_dict = {"Response": {"colname": "response", "format": "s"},
             "Ad Spend": {"colname": "cost", "format": "s"}}
tb = widgets.TabBar(list(plot_dict.keys()))
for k, v in plot_dict.items():
  with tb.output_to(k):
    # Set y-axis format
    format_value = ".1" if v["format"] == "s" else ".2"
    format_string = format_value + v["format"]

    # Set dataframe to use for plotting and define y-variable to plot
    plot_df = data_for_design[data_for_design["assignment"].isin(
        [group_control,
        group_treatment])].groupby(["date", "assignment"],
                                    as_index=False)[["response", "cost"]].sum()
    plot_df["assignment"] = plot_df["assignment"].map({
        group_control: "Control",
        group_treatment: "Treatment"
    })
    y_string = v["colname"] + ":Q"

    # Define how we select based on where the mouse is.
    selection = alt.selection_single(
        fields=["date"],
        nearest=True,
        on="mouseover",
        empty="none",
        clear="mouseout")
    bottom_selection = alt.selection_single(
        fields=["date"],
        nearest=True,
        on="mouseover",
        empty="none",
        clear="mouseout")

    # Brush for selecting a location to zoom into on x-axis.
    brush = alt.selection(type="interval", encodings=["x"])

    # Base chart -- same for top and bottom
    base = alt.Chart(plot_df).mark_line().encode(
        x=alt.X("date:T", axis=alt.Axis(title="", format=("%b %e")))
    )

    # Lines for data
    lines = base.mark_line().encode(
        y=alt.Y(y_string, axis=alt.Axis(title=" ", format=(format_string)))
    )
    bottom_lines = lines.encode(alt.X("date:T", scale=alt.Scale(domain=brush)))

    # Vertical rule
    rule = base.mark_rule().encode(
        opacity=alt.condition(selection, alt.value(0.3), alt.value(0)),
        tooltip=["date:T", y_string]
    ).add_selection(selection)
    bottom_rule = base.mark_rule().encode(
        opacity=alt.condition(bottom_selection, alt.value(0.3), alt.value(0)),
        tooltip=["date:T", y_string]
    ).add_selection(bottom_selection)

    # Points to denote where the mouse is.
    points = lines.mark_point().transform_filter(selection)
    bottom_points = bottom_lines.mark_point().transform_filter(bottom_selection)

    lines = lines.mark_line().encode(
      color=alt.Color("assignment", title=" "))
    bottom_lines = bottom_lines.mark_line().encode(
      color=alt.Color("assignment", title=" "))
    base_rule = base.transform_pivot(
        "assignment", value=v["colname"],
        groupby=["date"]).mark_rule().encode(tooltip=["date:T"] + [
            alt.Tooltip(c, type="quantitative")
            for c in plot_df["assignment"].unique()
        ])
    rule = base_rule.encode(
        opacity=alt.condition(selection, alt.value(0.3), alt.value(0))
    ).add_selection(selection)
    bottom_rule = base_rule.encode(
        opacity=alt.condition(bottom_selection, alt.value(0.3), alt.value(0))
    ).add_selection(bottom_selection)

    scatter_df = plot_df.pivot(
        index="date", values=v["colname"], columns="assignment").reset_index()
    base = alt.Chart(scatter_df).mark_circle(size=60).encode(
        alt.X("Control"),
        alt.Y("Treatment"),
        tooltip=["date", "Control", "Treatment"])
    regression_line = base.transform_regression(
        "Control", "Treatment", method="linear").mark_line(color="red")
    params = alt.Chart(scatter_df).transform_regression(
        'Control', 'Treatment', params=True
    ).mark_text(align='left', fontSize = 20).encode(
        x=alt.value(60),  # pixels from left
        y=alt.value(20),  # pixels from top
        text='rSquared:N'
    )
    string_r2 = alt.Chart({'values':[{'x': 30, 'y': 20}]}).mark_text(
      text='R^2 = ', fontSize = 20).encode( x=alt.value(30), y=alt.value(20))

    # Compile top chart
    top_chart = alt.layer(
        lines,
        points,
        rule
    ).add_selection(
        brush
    ).properties(
        width=chart_width,
        height=chart_height,
        title=k
    )

    # Compile bottom chart
    bottom_chart = alt.layer(
        bottom_lines,
        bottom_rule,
        bottom_points
    ).properties(
        width=chart_width,
        height=chart_height
    )

    # Compile scatterplot
    scatter_plot = alt.layer(base, regression_line, params, string_r2).properties(
        width=chart_width, height=chart_width, title=k).interactive()
    # Combine charts
    final_chart = alt.hconcat(
        alt.vconcat(
            top_chart,
            bottom_chart,
        ), scatter_plot).resolve_scale(color='independent')

    final_chart.configure_axis(
        titleFontSize=axis_title_font_size,
        labelFontSize=axis_label_font_size
    ).configure_title(
        fontSize=title_font_size
    ).display()



In [ ]:
#@title Summary and Results


print("Data in input:\n")
print("-  {} Geos \n".format(
    len(geo_level_time_series["geo"].unique())))

print("Output:\n")
print("The output contains two lists of geos: one for treatment" +
      " and the other for control\n")

human_baseline = utils.human_readable_number(baseline)
cost_baseline = list_of_budgets[selected_design] * 100 / baseline
print('-  {} Geos in treatment and {} geos control\n'.format(
    len(chosen_design.treatment_geos), len(chosen_design.control_geos)))
print("    Baseline store response: ${} for treatment\n".format(human_baseline))
print('    Cost/baseline = ${} / ${} ~ {:.3}%\n'.format(selected_budget,
                                                        human_baseline,
                                                        cost_baseline))

print(f'Minimum detectable iROAS = ' +
      f'{final_design["Minimum detectable iROAS"].values[0]}')
print(f'Minimum detectable lift in % = ' +
      f'{final_design["Minimum detectable lift in response"].values[0]}')

print(f"The design has Power {100 * power_level:.3}+% with Type-I error " +
      f"{100 *(1 - confidence_level):.3}% for testing H0: iROAS=0 vs " +
      f"H1: iROAS >= {final_design['Minimum detectable iROAS'].values[0]}")

In [ ]:
#@title Report stores for treatment and control separately and write to trix

#@markdown ###Insert the name google sheets in which we will save the data.
#@markdown The trix contains 4 worksheets, named:
#@markdown * "pretest data", containing the geo level time series;
#@markdown * "geopairs", containing the pairs of geos and their assignment.
#@markdown * "treatment geos", contains the list of geos in the treatment;
#@markdown * "control geos", contains the geos in the control groups.
Client_Name = "TBRMM_Client_Name" #@param {type:"string"}
filename_design = Client_Name + "_design.csv" #@param {type:"string"}

design_data = geo_level_time_series.copy()

design_data["assignment"] = "Excluded"
design_data.loc[design_data["geo"].isin(
    chosen_design.control_geos), "assignment"] = "Control"
design_data.loc[design_data["geo"].isin(
    chosen_design.treatment_geos), "assignment"] = "Treatment"

design_data["removed"] = False
design_data.loc[design_data["date"].isin(days_exclude), "removed"] = True

tmp_parameters = {
    "minimum_detectable_iROAS": minimum_detectable_iROAS,
    "average_order_value": average_order_value,
    "confidence_level": confidence_level,
    "power_level": power_level,
    "experiment_duration_in_weeks": experiment_duration_in_weeks,
    "experiment_budget": experiment_budget,
    "alternative_budget": alternative_budget,
    "day_week_exclude": ", ".join(day_week_exclude),
    "selected_design": selected_design,
}

parameters = {"parameter": list(tmp_parameters.keys()),
              "value": list(tmp_parameters.values())}


sh = gc.create(filename_design)
wid = sh.add_worksheet("pretest data", rows=1, cols=1)
set_with_dataframe(wid, design_data)
wid = sh.add_worksheet("treatment geos", rows=1, cols=1)
set_with_dataframe(wid, pd.DataFrame({"geo": list(chosen_design.treatment_geos)}))
wid = sh.add_worksheet("control geos", rows=1, cols=1)
set_with_dataframe(wid, pd.DataFrame({"geo": list(chosen_design.control_geos)}))
wid = sh.add_worksheet("parameters used in the design", rows=1, cols=1)
set_with_dataframe(wid, pd.DataFrame(parameters))
out = sh.del_worksheet(sh.sheet1)

# Appendix: